<p style="font-family: Arial; font-size:2.75em;color:purple; font-style:bold"><br>

Regresssion with scikit-learn<br><br> using Soccer Dataset

<br><br></p>


We will again be using the open dataset from the popular site <a href="https://www.kaggle.com">Kaggle</a> that we used in Week 1 for our example. 

Recall that this <a href="https://www.kaggle.com/hugomathien/soccer">European Soccer Database</a> has more than 25,000 matches and more than 10,000 players for European professional soccer seasons from 2008 to 2016. 

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Import Libraries<br><br></p>


In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import sqlite3
import numpy as np
import pandas as pd 

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Read Data from the Database into pandas
<br><br></p>


In [3]:
# Create your database connection and select
conn=sqlite3.connect('soccerbase.sqlite')
playerAtt=pd.read_sql_query("SELECT * FROM Player_Attributes",conn)

In [6]:
playerAtt.head()
playerAtt.shape
playerAtt.columns

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


(183978, 42)

Index(['id', 'player_fifa_api_id', 'player_api_id', 'date', 'overall_rating',
       'potential', 'preferred_foot', 'attacking_work_rate',
       'defensive_work_rate', 'crossing', 'finishing', 'heading_accuracy',
       'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy',
       'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
       'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina',
       'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
       'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
       'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning',
       'gk_reflexes'],
      dtype='object')

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Declare the Columns You Want to Use as Features
<br><br></p>


In [8]:
features = [
       'potential', 'crossing', 'finishing', 'heading_accuracy',
       'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy',
       'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
       'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina',
       'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
       'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
       'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning',
       'gk_reflexes']

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Specify the Prediction Target
<br><br></p>


In [9]:
target=['overall_rating']

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Clean the Data<br><br></p>


In [7]:
playerAtt=playerAtt.dropna()

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Extract Features and Target
<br><br></p>


In [10]:
X=playerAtt[features]

In [11]:
y=playerAtt[target]

Let us look at a typical row from our features: 

In [12]:
X.iloc[2]

potential             66.0
crossing              49.0
finishing             44.0
heading_accuracy      71.0
short_passing         61.0
volleys               44.0
dribbling             51.0
curve                 45.0
free_kick_accuracy    39.0
long_passing          64.0
ball_control          49.0
acceleration          60.0
sprint_speed          64.0
agility               59.0
reactions             47.0
balance               65.0
shot_power            55.0
jumping               58.0
stamina               54.0
strength              76.0
long_shots            35.0
aggression            63.0
interceptions         41.0
positioning           45.0
vision                54.0
penalties             48.0
marking               65.0
standing_tackle       66.0
sliding_tackle        69.0
gk_diving              6.0
gk_handling           11.0
gk_kicking            10.0
gk_positioning         8.0
gk_reflexes            8.0
Name: 2, dtype: float64

Let us also display our target values: 

In [ ]:
y

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Split the Dataset into Training and Test Datasets
<br><br></p>


In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.33,random_state=23)

In [15]:
X_train.shape
X_test.shape

(120837, 34)

(59517, 34)

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

(1) Linear Regression: Fit a model to the training set
<br><br></p>


In [16]:
from sklearn.linear_model import LinearRegression
regressor=LinearRegression()

In [17]:
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [20]:
regressor.coef_
regressor.coef_.shape
regressor.intercept_

array([[ 0.37906686,  0.02357643,  0.01081523,  0.06920192,  0.05220679,
         0.00438408, -0.01288934,  0.01130752,  0.01323301,  0.00551949,
         0.13197915,  0.00597854,  0.0089733 , -0.00872732,  0.21063184,
         0.00767495,  0.01699448,  0.01607134, -0.0057261 ,  0.0609912 ,
        -0.01310573,  0.02076542,  0.01283421, -0.01029265, -0.00184794,
         0.01426678,  0.03344572,  0.00235897, -0.02888893,  0.16384081,
         0.03453613, -0.03365246,  0.0522556 ,  0.02516012]])

(1, 34)

array([-3.55769444])

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Perform Prediction using Linear Regression Model
<br><br></p>


In [22]:
y_prediction=regressor.predict(X_test)
y_prediction
y_prediction.shape

array([[ 75.09836586],
       [ 68.15402864],
       [ 67.28737419],
       ..., 
       [ 71.84008323],
       [ 63.28038771],
       [ 57.63124778]])

(59517, 1)

In [23]:
predictedy=np.concatenate([y_prediction,X_test],axis=1)
predictedy=pd.DataFrame(predictedy)

In [24]:
type(predictedy)

pandas.core.frame.DataFrame

In [25]:
predictedy.head()

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,75.098366,75.0,57.0,75.0,84.0,70.0,76.0,64.0,80.0,51.0,...,62.0,61.0,17.0,24.0,30.0,12.0,5.0,7.0,5.0,7.0
1,68.154029,72.0,60.0,66.0,58.0,68.0,59.0,70.0,67.0,36.0,...,72.0,54.0,23.0,38.0,36.0,12.0,10.0,12.0,13.0,10.0
2,67.287374,70.0,66.0,32.0,64.0,58.0,35.0,65.0,52.0,45.0,...,48.0,49.0,71.0,75.0,73.0,8.0,12.0,11.0,8.0,15.0
3,57.472677,59.0,53.0,48.0,46.0,52.0,64.0,50.0,61.0,61.0,...,56.0,56.0,25.0,25.0,25.0,11.0,6.0,6.0,7.0,14.0
4,65.506230,75.0,64.0,32.0,51.0,59.0,34.0,66.0,41.0,35.0,...,39.0,50.0,69.0,69.0,71.0,9.0,13.0,16.0,14.0,6.0


In [26]:
y_test.describe()

,overall_rating
count,59517.000000
mean,68.662785
std,7.043052
min,35.000000
25%,64.000000
50%,69.000000
75%,73.000000
max,94.000000


In [27]:
predictedy.describe()

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
count,59517.000000,59517.000000,59517.000000,59517.000000,59517.000000,59517.000000,59517.000000,59517.000000,59517.000000,59517.000000,...,59517.000000,59517.00000,59517.000000,59517.000000,59517.000000,59517.000000,59517.000000,59517.000000,59517.000000,59517.000000
mean,68.654541,73.509703,55.201673,50.081809,57.180486,62.463212,49.508309,59.345347,53.008334,49.387049,...,57.902918,54.91547,46.660349,50.267772,47.913974,14.742359,15.993262,20.602433,16.068737,16.379287
std,6.469229,6.605230,17.319202,19.065383,16.507097,14.251078,18.311549,17.812315,18.307408,17.873054,...,15.202970,15.57943,21.223431,21.508652,21.631654,16.914830,15.895357,21.207692,16.136010,17.248095
min,37.131419,39.000000,1.000000,1.000000,1.000000,3.000000,1.000000,1.000000,4.000000,1.000000,...,1.000000,3.00000,1.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,64.410708,69.000000,45.000000,34.000000,49.000000,57.000000,35.000000,52.000000,41.000000,36.000000,...,49.000000,45.00000,25.000000,29.000000,25.000000,7.000000,8.000000,8.000000,8.000000,8.000000
50%,68.698181,74.000000,59.000000,53.000000,60.000000,65.000000,52.000000,64.000000,56.000000,50.000000,...,60.000000,57.00000,50.000000,56.000000,53.000000,10.000000,11.000000,12.000000,11.000000,11.000000
75%,73.150029,78.000000,68.000000,66.000000,68.000000,72.000000,64.000000,72.000000,67.000000,64.000000,...,69.000000,67.00000,66.000000,69.000000,67.000000,13.000000,15.000000,15.000000,15.000000,15.000000
max,92.434503,96.000000,94.000000,97.000000,98.000000,97.000000,93.000000,97.000000,94.000000,96.000000,...,97.000000,96.00000,94.000000,94.000000,95.000000,93.000000,92.000000,95.000000,94.000000,94.000000


<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

What is the mean of the expected target value in test set ?
<br><br></p>


<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Evaluate Linear Regression Accuracy using Root Mean Square Error

<br><br></p>


In [29]:
from math import sqrt
from sklearn.metrics import mean_squared_error
RMSElinear=sqrt(mean_squared_error(y_true=y_test,y_pred=y_prediction))

In [30]:
print(RMSElinear)

2.7835912906853975


In [32]:
#Practice: Caculate the RMSE for the training set
y_predtrain=regressor.predict(X_train)
RMSEtrain=sqrt(mean_squared_error(y_true=y_train,y_pred=y_predtrain))
print(RMSEtrain)

2.8024651264902696


<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

K-folds cross-validation

<br><br></p>

In [33]:
from sklearn import metrics
from sklearn.model_selection import cross_val_score
scores=cross_val_score(regressor, X, y, cv=5)
scores

array([ 0.84388689,  0.83717095,  0.84051477,  0.84019942,  0.8425994 ])

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

(2) Decision Tree Regressor: Fit a new regression model to the training set
<br><br></p>


In [34]:
from sklearn.tree import DecisionTreeRegressor
regressorT = DecisionTreeRegressor(max_depth=20)
regressorT.fit(X_train,y_train)

DecisionTreeRegressor(criterion='mse', max_depth=20, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best')

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Perform Prediction using Decision Tree Regressor
<br><br></p>


In [35]:
y_predT=regressorT.predict(X_test)

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

For comparision: What is the mean of the expected target value in test set ?
<br><br></p>


<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Evaluate Decision Tree Regression Accuracy using Root Mean Square Error

<br><br></p>


In [37]:
RMSEtree=sqrt(mean_squared_error(y_true=y_test,y_pred=y_predT))

In [38]:
print(RMSEtree)

1.4757519304556492


In [39]:
scoresT=cross_val_score(regressorT, X, y, cv=5)
scoresT

array([ 0.88466361,  0.88293773,  0.88549672,  0.88440301,  0.88959242])